In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

# Import our input dataset
tele_df = pd.read_csv('bank_telemarketing.csv')
tele_df.head()

,Age,Job,Marital_Status,Education,Default_Credit,Housing_Loan,Personal_Loan,Subscribed
0,56,other,married,Primary_Education,no,no,no,no
1,37,services,married,Secondary_Education,no,yes,no,no
2,40,admin,married,Primary_Education,no,no,no,no
3,56,services,married,Secondary_Education,no,no,yes,no
4,59,admin,married,Professional_Education,no,no,no,no


In [2]:
# Generate our categorical variable list
tele_cat = tele_df.dtypes[tele_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
tele_df[tele_cat].nunique()

Job               9
Marital_Status    3
Education         4
Default_Credit    2
Housing_Loan      2
Personal_Loan     2
Subscribed        2
dtype: int64

In [3]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(tele_df[tele_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(tele_cat)
encode_df.head()

,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,Marital_Status_divorced,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [4]:
# Merge one-hot encoded features and drop the originals
tele_df = tele_df.merge(encode_df,left_index=True, right_index=True)
tele_df = tele_df.drop(tele_cat,1)
tele_df.head()

,Age,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [5]:
# Remove loan status target from features data
y = tele_df.Subscribed_yes.values
X = tele_df.drop(columns=["Subscribed_no","Subscribed_yes"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
# Create the SVM model
svm = SVC(kernel='linear')

In [7]:
# Train the model
svm.fit(X_train, y_train)

SVC(kernel='linear')

In [8]:
# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 SVM model accuracy: 0.873


In [9]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [10]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
715/715 [==============================] - 1s 377us/step - loss: 0.4066 - accuracy: 0.8703
Epoch 2/50
715/715 [==============================] - 0s 379us/step - loss: 0.3727 - accuracy: 0.8735
Epoch 3/50
715/715 [==============================] - 0s 381us/step - loss: 0.3703 - accuracy: 0.8735
Epoch 4/50
715/715 [==============================] - 0s 378us/step - loss: 0.3696 - accuracy: 0.8735
Epoch 5/50
715/715 [==============================] - 0s 381us/step - loss: 0.3687 - accuracy: 0.8735
Epoch 6/50
715/715 [==============================] - 0s 383us/step - loss: 0.3683 - accuracy: 0.8735
Epoch 7/50
715/715 [==============================] - 0s 379us/step - loss: 0.3679 - accuracy: 0.8735
Epoch 8/50
715/715 [==============================] - 0s 378us/step - loss: 0.3676 - accuracy: 0.8735
Epoch 9/50
715/715 [==============================] - 0s 380us/step - loss: 0.3673 - accuracy: 0.8735
Epoch 10/50
715/715 [==============================] - 0s 381us/step - loss: 0.367

In [11]:
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                240       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
715/715 [==============================] - 0s 375us/step - loss: 0.3621 - accuracy: 0.8734

Epoch 00001: saving model to checkpoints\weights.01.hdf5
Epoch 2/100
715/715 [==============================] - 0s 371us/step - loss: 0.3618 - accuracy: 0.8736

Epoch 00002: saving model to checkpoints\weights.02.hdf5
Epoch 3/100
715/715 [==============================] - 0s 381us/step - loss: 0.3619 - accuracy: 0.8736

Epoch 00003: saving model to checkpoints\weights.03.hdf5
Epoch 4/100
715/715 [==============================] - 0s 373us/step - loss: 0.3620 - accuracy: 0.8734

Epoch 00004: saving model to checkpoints\weights.04.hdf5
Epoch 5/100
715/715 [==============================] - 0s 377us/step - loss: 0.3619 - accuracy: 0.8737

Epoch 00005: saving model to checkpoints\weights.05.hdf5
Epoch 6/100
715/715 [==============================] - 0s 378us/step - loss: 0.3620 - accuracy: 0.8732

Epoch 00006: saving model to checkpoints\weights.06.hdf5
Epoch 7/100
715/715 [=============


Epoch 00051: saving model to checkpoints\weights.51.hdf5
Epoch 52/100
715/715 [==============================] - 0s 381us/step - loss: 0.3604 - accuracy: 0.8732

Epoch 00052: saving model to checkpoints\weights.52.hdf5
Epoch 53/100
715/715 [==============================] - 0s 383us/step - loss: 0.3604 - accuracy: 0.8736

Epoch 00053: saving model to checkpoints\weights.53.hdf5
Epoch 54/100
715/715 [==============================] - 0s 380us/step - loss: 0.3605 - accuracy: 0.8738

Epoch 00054: saving model to checkpoints\weights.54.hdf5
Epoch 55/100
715/715 [==============================] - 0s 381us/step - loss: 0.3603 - accuracy: 0.8735

Epoch 00055: saving model to checkpoints\weights.55.hdf5
Epoch 56/100
715/715 [==============================] - 0s 372us/step - loss: 0.3603 - accuracy: 0.8733

Epoch 00056: saving model to checkpoints\weights.56.hdf5
Epoch 57/100
715/715 [==============================] - 0s 378us/step - loss: 0.3603 - accuracy: 0.8735

Epoch 00057: saving model t

In [15]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

239/239 - 0s - loss: 0.3705 - accuracy: 0.8727
Loss: 0.37050318717956543, Accuracy: 0.8727034330368042


In [16]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [17]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [18]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

239/239 - 0s - loss: 0.3705 - accuracy: 0.8727
Loss: 0.37050318717956543, Accuracy: 0.8727034330368042
